In [ ]:
스타벅스 크롤링 & 분석

In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
import pandas as pd

In [3]:

url = 'https://www.starbucks.co.kr/store/store_map.do'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

C:\Users\JUNG SO YOUNG\AppData\Local\Temp\ipykernel_14380\3232498481.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [ ]:
area_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element(By.CSS_SELECTOR,area_btn).click()

time.sleep(2)
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR,seoul_btn).click()

time.sleep(2)
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR,all_btn).click()

In [5]:
txt = driver.page_source
html = bs(txt)

len(html.select('li'))

426

In [6]:
len(html.select('li.quickResultLstCon'))

10

In [ ]:
shops = html.select('li.quickResultLstCon')

In [ ]:
txt = driver.page_source
html = bs(txt)

len(html.select('li'))

shops = html.select('li.quickResultLstCon')

shop = shops[0]
shop

In [ ]:
# 위도 : data-lat
lat = shop['data-lat']
# 경도 : data-long
lng = shop['data-long']
# 지점명 : data-name
name = shop['data-name']

# 주소 : 딕셔너리 형태가 아니기 때문에 select으로 추출
addr = str(shop.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]

# 전화번호 : 주소와 전화번호는 묶여 있으므로 복잡하게 추출
tel = str(shop.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]


# 매장종류 : 딕셔너리 형태가 아니기 때문에 select으로 추출
shoptype = shop.select('i.pin_general')[0].text

print(lat)
print(lng)
print(name)
print(addr)
print(tel)
print(shoptype)

In [ ]:
['매장명', '위도', '경도', '주소', '전화번호', '매장종류']

starbucks_list = []

for shop in shops:
    lat = shop['data-lat']
    lng = shop['data-long']
    name = shop['data-name']
    addr = str(shop.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(shop.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    stype = shop.select('i')[0].text

    time.sleep(0.01)
    starbucks_list.append([name, lat, lng, addr, tel, stype])

print(starbucks_list)

In [ ]:
df = pd.DataFrame(starbucks_list, columns=['매장명', '위도', '경도', '주소', '전화번호', '매장종류'])
df

In [ ]:
df.to_excel('starbucks_seoul.xlsx', index=False)

df = pd.read_excel('starbucks_seoul.xlsx', header=0)

In [ ]:
##########################################
#                데이터 분석
##########################################

# !pip install folium

In [ ]:
import folium

starmap = folium.Map()

starmap

In [ ]:
import folium

starmap = folium.Map(
    location=[37.5666805, 126.9784147],   # 구글에서 검색한 서울 시청 좌표
    zoom_start=11
)

starmap

In [ ]:
import folium

starmap = folium.Map(
    location=[37.5666805, 126.9784147],   # 구글에서 검색한 서울 시청 좌표
    zoom_start=11,
    tiles='Stamen Watercolor'
)

# 타일 종류
# OpenStreetMap (기본값)
# Stamen Terrain, Stamen Toner, Stamen Watercolor
# CartoDB positron, CartoDB dark_matter
starmap

In [ ]:
import folium

starmap = folium.Map(
    location=[37.5666805, 126.9784147],   # 구글에서 검색한 서울 시청 좌표
    zoom_start=11,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[37.5666805, 126.9784147],
    fill = True
).add_to(starmap)

starmap

In [ ]:
import folium

starmap = folium.Map(
    location=[37.5666805, 126.9784147],   # 구글에서 검색한 서울 시청 좌표
    zoom_start=11,
    tiles='Stamen Terrain'
)

# 데이터 프레임에서 위도와 경도 값을 가져와서 600번 반복

for idx in df.index:
    lat = df.loc[idx, '위도']
    lng = df.loc[idx, '경도']


folium.Marker(
    location=[lat, lng],
    fill = True
).add_to(starmap)

starmap

In [ ]:
# 각 지점에서 '구'명 추출

gu_names = []

for addr in df['주소']:
    gu = addr.split()[1]
    gu_names.append(gu)

print(gu_names)
print(len(gu_names))

In [ ]:
df['시군구명'] = gu_names
df

In [ ]:
# 구별 스타벅스 매장 수 계산

starbucks_count = df.pivot_table(index='시군구명', 
                                 values='매장명',
                                 aggfunc='count').rename(columns={'매장명':'매장수'})

starbucks_count

In [ ]:
# 거주자 수, 업체 수 등의 엑셀 파일과 결합하기

seoul_sgg = pd.read_excel('seoul_sgg_stat.xlsx', thousands=',')
seoul_sgg

In [ ]:
seoul_sgg = pd.merge(seoul_sgg, starbucks_count, how='left', on='시군구명')
seoul_sgg

In [ ]:
# 주민등록인구 십만명당 매장수

seoul_sgg['거주자_십만명당_매장수'] = seoul_sgg['매장수'] / (seoul_sgg['주민등록인구']/100000)